# Stability of timestepping schemes

You already saw in CMIII that for some problems if we make the timestep too large in the forward Euler method, we don't get a valid solution. Let's remind ourselves with an example.

In [ ]:
%matplotlib notebook
import numpy
from matplotlib import pyplot
pyplot.style.use("ggplot")

from scipy.linalg import expm

class linear(object):
    def __init__(self, A):
        self.A = A.copy()
    
    def __call__(self, t, u):
        return self.A @ u
    
    def exact(self, t, u_0):
        t = numpy.array(t, ndmin=1)
        return [numpy.real_if_close(expm(self.A*s) @ u_0) for s in t]

test = linear(numpy.array([[0, 1],
                           [-1, 0]]))
u_0 = numpy.array([.75, 0])

def ode_euler(f, u_0, h=0.1, T=1):
    u = numpy.array(u_0)
    t = 0
    thist = [t]
    uhist = [u_0]
    while t < T:
        h = min(h, T - t)
        u = u + h * f(t, u)
        t += h
        thist.append(t)
        uhist.append(u.copy())
    return numpy.asarray(thist), numpy.asarray(uhist)

pyplot.figure()

k = 2
for h in [0.01, 0.1, 0.5]:
    thist, uhist = ode_euler(test, u_0, h=h, T=15)
    pyplot.plot(thist, uhist, "o", linestyle="solid", label=f"Forward Euler h={h}")
pyplot.plot(thist, test.exact(thist, u_0), label='Exact')
pyplot.legend(bbox_to_anchor=(0.5, 1), loc='center', ncol=2);

Why is this?

To answer this, we consider the behaviour of the integrator on a linear test problem, the *Dahlquist test equation*:

$$
\dot u = \lambda u \quad \lambda \in \mathbb{C}.
$$

When taking a step of length $h$, this equation has the exact solution

$$
u(h) = u_0 e^{\overbrace{\lambda h}^z} = u_0 e^{\mathfrak{R} z}(\cos \mathfrak{I} z + i\sin\mathfrak{I} z),
$$
where we wrote $\mathbb{C} \ni z = \lambda h$.

This problem is _physically_ stable whenever the real part of $z$ (and hence $\lambda$) is less than zero: $\mathfrak{R} z \le 0$.

### Aside

There is also theory surrounding _nonlinear_ stability, but we will not cover it in this course.

## Stability regions

Let us now consider applying the timestepping schemes we've already encountered to the test equation to see how they behave.

### Explicit Euler

$$
u(h) = \underbrace{(1 + \lambda h)}_{R(z)}u_0.
$$

Repeated application of the scheme results in:

$$
u(mh) = R(z)^m u_0.
$$

This scheme is convergent (producing a bounded $u(mh)$ in the limit $m \to \infty$) only if

$$
|R(z)| \le 1.
$$

### Implicit Euler

Performing a similar calculation we obtain

$$
R(z) = \frac{1}{1 - z}.
$$

### Trapezoidal rule/implicit midpoint

$$
R(z) = \frac{1 + z/2}{1 - z/2}.
$$

### Definitions
The function $R(z)$ is called the *stability function* of the method, the set

$$
S = \{ z \in \mathbb{C} \colon |R(z)| \le 1 \}
$$

is called the *stability domain*.

## Stability plots

Visualising the stability domains, by plotting $|R(z)|$ is an insightful way of comparing methods:

In [ ]:
%matplotlib notebook
from matplotlib import pyplot
import numpy

def plot_stability(x, y, R, label):
    pyplot.figure()
    C = pyplot.contourf(x, y, numpy.abs(R), numpy.linspace(0, 1, 10), cmap=pyplot.cm.coolwarm)
    
    pyplot.colorbar(C, ticks=numpy.linspace(0, 1, 10))
    pyplot.contour(x, y, numpy.abs(Rz), numpy.linspace(0, 1,4), colors='k')
    pyplot.title(label)

In [ ]:
x = numpy.linspace(-3, 3)
x, y = numpy.meshgrid(x, x)
z = x + 1j*y

Rs = [("Forward Euler", 1 + z),
      ("Backward Euler", 1/(1 - z)),
      ("Implicit Midpoint", (1 + z/2)/(1 - z/2))]

for label, Rz in Rs:
    plot_stability(x, y, Rz, label)

### Observations

While the physical problem is stable whenever $\mathfrak{R} \lambda \le 0$, the same is not true of all our methods. The explicit Euler method is only stable in a small region of the left half plane, implicit Euler is stable in the entire left half plane, and more, implicit midpoint is stable in exactly the left half plane.

A question naturally arises? Why would one ever use an explicit method? Can you think of any reasons?

## Definitions

### A-stability

A method is _A-stable_ if the stability domain

$$
S = \{z \in \mathbb{C} \colon |R(z)| \le 1\}
$$

contains the _entire_ left half plane

$$
\mathfrak{R} z \le 0.
$$

This means that we can take arbitrarily large timesteps ($h \to \infty$) without the method becoming unstable (diverging) for any problem that is physically stable. Note that this says nothing about the _accuracy_ of the method.

### Questions

1. Show that the midpoint method and implicit Euler really do contain the entire left half plane. Hint: multiply through by an appropriate choice of $1$, write $z = a + bi$ and show that $|R(z)| \le 1$ whenever $a \le 0$.

## Higher order methods

So far, we've seen the implicit and explicit Euler methods which are only first order accurate, and the implicit midpoint method which is second order accurate. We might, especially if we have an accurate spatial discretisation, want more accuracy in our time integrator. We will now look at some methods that offer this.

In this section we will look at both single-step Runge-Kutta methods (covered briefly at the very end of CMIII) and multi-step methods.

Many of these methods offer a clever way of estimating the local error accumulation at each step, and we will look at how this can be used to implement an adaptive timestepping scheme.

High-order timestepping schemes also have a higher cost of evaluation (as we shall see), and we'll discuss how to compare different schemes in a fair way using _work-precision_ diagrams.

## Runge-Kutta methods

These are a class of _single-step_ methods. That is, we advance a solution $u_0$ from $t_0$ to $t_1$ using $u_0$ as an initial value, producing $u_1$, then when advancing from $t_1$ to $t_2$ we forget about $u_0$ and use $u_1$ as an initial value. We will look at _multi-step_ methods in a short while.

Recall that the single-step methods we have looked at so far (Euler and midpoint schemes) are all obtained by formally solving the ODE with integration and then approximating the integral.

$$
\dot u = f(t, u), u(t_0) = u_0
$$

and hence

$$
u(t) = u(t_0) + \int_{t_0}^t f(\tau, u) \text{d}\tau.
$$

A single step is then obtained by setting $t = t_0 + h$ and writing

$$
\int_{t_0}^{t_0 + h} f(\tau, u) \text{d}\tau = h\int_0^1 f(t_0 + h\tau, y(t_0 + h\tau))\text{d}\tau.
$$

We now _approximate_ the integral with a sum (using a quadrature scheme, but let's sweep that under the carpet for now), by picking a set of points $\{\xi_i\}_{i=1}^{N} \in [0, 1]$ and weights $\{w_i\}_{i=1}^{N} \in \mathbb{R}$

$$
\int_0^1 f(t_0 + h\tau, y(t_0 + h\tau))\text{d}\tau \approx \sum_{i=1}^{N} w_i f(t_0 + \xi_i h, y(t_0 + \xi_i h)).
$$

Giving us our update formula:
$$
u_{n+1} = u_n + h \sum_{i=1}^{N} w_i f(t_n + \xi_i h, u(t_n + \xi_i h)).
$$

For example, explicit Euler was obtained with $N=1$, $\{\xi_i\} = \{0\}$, and $\{w_i\} = \{1\}$.

Our challenge, for more general schemes, is to figure out how to approximate the (unknown) $y$s, we only know the initial value $y(t_0)$.

### Explicit Runge-Kutta methods

Write $Y_i$ for our numerical approximation to $u(t_0 + \xi_i h)$. _Explicit_ Runga-Kutta methods compute $Y_i, i = 2,\dots,N$ as a linear combination of $Y_j, j < i$.

That is, we write

$$
\begin{align}
Y_1 &= u_n\\
Y_2 &= u_n + h a_{2, 1} f(t_n, Y_1)\\
Y_3 &= u_n + h(a_{3,1} f(t_n, Y_1) + a_{3,2} f(t_n + c_2 h, Y_2)\\
&\vdots\\
Y_N &= u_n + h \sum_{i=1}^{N-1} a_{N, i} f(t_n + c_i h, Y_i)\\
u_{n+1} &= u_n + h \sum_{i=1}^{N} b_{i} f(t_n + c_i h, Y_i).
\end{align}
$$

Each of these intermediate steps is termed a _stage_, and so we just wrote down an $N$-stage Runge-Kutta method.

For an explicit method, the _RK matrix_ (or *coefficients*)

$$
A = \begin{bmatrix}
a_{1, 1} & a_{1, 2} & \dots & a_{1, N}\\
a_{2, 1} & a_{2, 2} & \dots & a_{2, N}\\
\vdots & \vdots & \ddots & \vdots\\
a_{N, 1} & a_{N, 2} & \dots & a_{N, N}\\
\end{bmatrix}
$$

is strictly lower-triangular ($a_{i, j} = 0, j \ge i$).

If we further write the _RK weights_ (or *completion weights*)

$$
b = \begin{bmatrix}
b_1 \\
b_2 \\
\vdots \\
b_N
\end{bmatrix}
$$

and the _RK nodes_ (or *abscissa*)

$$
c = \begin{bmatrix}
c_1 \\
c_2 \\
\vdots \\
c_N
\end{bmatrix}
$$

the entire scheme can be compactly represented in a _Butcher tableau_ 

$$
\begin{array}{c|c}
c & A\\
\hline
 & b^T
\end{array}
$$

(named after [John Butcher](https://en.wikipedia.org/wiki/John_C._Butcher) who developed much of the theory underpinning the analysis and design of Runge-Kutta methods)


### Implict Runge-Kutta methods

We have already seen that we will want an implicit, rather than explicit, method when taking large timesteps. The same formalism for expressing Runge-Kutta methods is also applicable to implicit ones. In this case, $A$ is no longer strictly lower-triangular. Popular implicit RK schemes are typically at least lower triangular ($a_{i,j} = 0, j > i$). This way, although we need to solve a system for each stage, we can still do forward-substitutions, and the stage equations become:

$$
Y_i - h a_{i,i} f(t_n + c_i h, Y_i) = u_n + h \sum_{j < i} a_{i, j} f(t_n + c_j h, Y_j).
$$


These are termed DIRK (_diagonally_ implicit Runge-Kutta) methods. 

A subclass that is even better (if achievable) is SDIRK (_singly_ diagonally implicit Runge-Kutta) methods. This last implicit class has $a_{i,i} = \alpha$ for all $i$.

These methods are attractive because we can often reuse (part of) the setup cost for solving the systems at each stage.

### Determining $A$, $b$, and $c$

For low order methods, our first thought is to Taylor expand everything around $(t_n, u_n)$ and then equate terms with the Taylor expansion of the exact solution.

This is doable for $N=2$, but already becomes tremendously labourious at $N=3$. Fortunately, other people (notably Butcher) have developed graph theoretical tools to construct, and analyse the properties of, Runge-Kutta methods. Here, we will only use them.

One particular point to note is that the conditions on the entries in $A$, $b$, and $c$ do not uniquely specify the coefficients, so it is possible to come up with multiple different methods with the same number of stages.

After all that, let's write some code to do explict RK integration.

In [ ]:
class ButcherTable(object):
    def __init__(self, A, b, btilde=None):
        self.A = numpy.asarray(A)
        self.b = numpy.asarray(b)
        self.btilde = numpy.asarray(btilde) if btilde else None
        self.c = self.A.sum(axis=1)

    def _repr_latex_(self):
        rows, cols = self.A.shape
        strs = ["$$", r"\left[", r"\begin{array}{c|%s}" % ("c"*cols)]
        for r in range(rows):
            row = " & ".join(map(str, [self.c[r]] + list(self.A[r, :])))
            strs.append(row)
            strs.append(r"\\")
        strs.append(r"\hline")
        
        strs.append("&")
        strs.append(" & ".join(map(str, self.b)))
        strs.append(r"\\")
        if self.btilde is not None:
            strs.append("&" + " & ".join(map(str, self.btilde)))
        strs.extend([r"\end{array}", r"\right]", "$$"])
        return "\n".join(strs)

def ode_rk_explicit(f, u_0, table, h=0.1, T=1):
    A = table.A
    b = table.b
    c = A.sum(axis=1)
    N = len(c)
    t = 0
    u_0 = numpy.asarray(u_0)
    u = u_0.copy()
    hist = [(t, u.copy())]
    fY = numpy.zeros(u_0.shape + (N, ))
    while t < T:
        h = min(h, T - t)
        fY[:] = 0
        for i in range(N):
            Yi = u.copy()
            for j in range(i):
                Yi += h * A[i, j] * fY[:, j]
            fY[:, i] = f(t + h*c[i], Yi)
        u += h * fY @ b
        t += h
        hist.append((t, u.copy()))
    ts, us = zip(*hist)
    return numpy.asarray(ts), numpy.asarray(us)

In [ ]:
euler = ButcherTable([[0]], [1])

In [ ]:
euler

In [ ]:
ts, us = ode_rk_explicit(test, u_0, euler, h=0.1, T=15)
pyplot.figure()
pyplot.plot(ts, us);

We can also combine multiple explicit Euler steps for a higher order method, [Heun's method](https://en.wikipedia.org/wiki/Heun%27s_method) (also called "modified Euler" or "explicit trapezoid"). This has Butcher tableau

$$
\left[
\begin{array}{c|cc}
0 & 0 & 0\\
1 & 1 & 0\\
\hline
  & 1/2 & 1/2
\end{array}
\right]
$$

In [ ]:
heun = ButcherTable(numpy.asarray([[0, 0], [1, 0]]), numpy.asarray([1/2, 1/2]))
ts, us = ode_rk_explicit(test, u_0, heun, h=0.1, T=15)
pyplot.figure()
pyplot.plot(ts, us);

Perhaps _the_ most famous RK method is the explicit 4-th order RK4

In [ ]:
rk4 = ButcherTable(numpy.asarray([[0, 0, 0, 0],
                                  [1/2, 0, 0, 0],
                                  [0, 1/2, 0, 0],
                                  [0, 0, 1, 0]]),
                   numpy.asarray([1/6, 1/3, 1/3, 1/6]))
rk4

In [ ]:
ts, us = ode_rk_explicit(test, u_0, rk4, h=0.1, T=15)
pyplot.figure()
pyplot.plot(ts, us);

## Questions

1. Derive an expression for the stability function $R(z)$ for RK methods by applying one step to the Dahlquist test equation
2. Extend the `ButcherTable` class with a method to plot the method's stability region.
3. What do you see when comparing the stability of Heun and RK4 to that of explicit Euler?

## Error control

In CMIII you saw a relatively simple method for adaptive timestepping. Given a single timestepping scheme with step length $h$, the error can be estimated by performing two steps with step length $h/2$ and computing the difference in the resulting guess.

This is rather wasteful, in that we need to perform three time integrations to take one step.

For higher-order RK schemes, a more sophisticated approach is possible. Since the constraints on the coefficients in the Butcher table can admit more than one possible solution it is often possible to use the same set of stages with two different sets of *completion weights* (the $b$ vectors) to produce two updates of different order effectively "for free". The error can then be estimated by computing the difference between the two steps.

### Error estimation with embedded methods

Let us suppose we have an RK method

$$
\left[
\begin{array}{c|c}
c & A\\
\hline
& b^T\\
& \tilde{b}^T
\end{array}
\right]
$$

Where the completion weights $b^T$ produce a method of order $p$, and the weights $\tilde{b}^T$ produce a mthod of order $\tilde{p} = p - 1$. We then have, from a starting point $u_n$, two candidate solutions:

$$
\begin{align}
u_{n+1} &= u_n + h b f(Y)\\
\tilde{u}_{n+1} &= u_n + h b f(Y)\\
\end{align}
$$

where $f(Y)$ is the vector of intermediate function evaluations.

We can now estimate the error in a single step by

$$
e(h) = \| u_{n+1} - \tilde{u}_{n+1} \| = \| h (b - \tilde{b})^T f(Y) \| \in \mathcal{O}(h^p)
$$

Given some desired tolerance $\epsilon$, we want to find $h_*$ such that

$$
e(h_*) < \epsilon
$$

Since $e(h) \in \mathcal{O}(h^p)$ we can write $e(h) = c h^p$, then our requirement on $h_*$ means we want

$$
c h_*^p < \epsilon
$$

and so

$$
h_* < \left(\frac{\epsilon}{c}\right)^{1/p}.
$$

We estimate the unknown $c$ by using our error estimate:

$$
e(h) = c h^p \Leftrightarrow c = \frac{e(h)}{h^p}
$$

and substituting in

$$
h^* < h \left(\frac{\epsilon}{e(h)}\right)^{1/p}
$$

### BS3

A low-order method which implements this embedded idea is the [Bogacki-Shampine](https://en.wikipedia.org/wiki/Bogacki–Shampine_method) method of order 3, which comes with an embedded 2nd order method.

In [ ]:
bs3 = ButcherTable([[0, 0, 0, 0],
                    [1/2, 0, 0, 0],
                    [0, 3/4, 0, 0],
                    [2/9, 1/3, 4/9, 0]],
                  [2/9, 1/3, 4/9, 0], # order 3
                  btilde=[7/24, 1/4, 1/3, 1/8] # order 2
                  )

Although this method has 4 stages, it only requires 3 function evaluations, since if we stare closely we can see that the last stage has the same coefficients as the order 3 completion formula (so we can reuse the last stage for the next timestep). This is termed the _First Same as Last_ or FSAL property.

In [ ]:
bs3

### ODE45

The default ODE solver in MATLAB (amongst others) is the 5th order [Dormand-Prince](https://en.wikipedia.org/wiki/Dormand–Prince_method) method (with a 4th order embedded method). One can vaguely imagine that this table was not computed by frantically Taylor expanding!

In [ ]:
dp45 = ButcherTable([[0, 0, 0, 0, 0, 0, 0],
                     [1/5, 0, 0, 0, 0, 0, 0],
                     [3/40, 9/40, 0, 0, 0, 0, 0],
                     [44/45, -56/15, 32/9, 0, 0, 0, 0],
                     [19372/6561, -25360/2187, 64448/6561, -212/729, 0, 0, 0],
                     [9017/3168, -355/33, 46732/5247, 49/176, -5103/18656, 0, 0],
                     [35/384, 0, 500/1113, 125/192, -2187/6784, 11/84, 0]],
                   [35/384, 0, 500/1113, 125/192, -2187/6784, 11/84, 0], # 5th order
                   btilde=[5179/57600, 0, 7571/16695, 393/640, -92097/339200, 187/2100, 1/40] # 4th order
                   )
dp45

Note how this method also has the FSAL property for the 5th order completion formula.

## Comparing methods

All other things being equal, a higher-order method is likely a superior choice, since it will converge faster to the exact solution with bigger timesteps. For example, let's compute the convergence rate for the explicit Euler, BS3, RK4, and DP5 methods on our linear osciliatory test problem.

In [ ]:
from functools import partial

def mms(eqn, integrator, u_0, h=0.1, T=15):
    thist, uhist = integrator(eqn, u_0, h=h, T=T)
    return numpy.linalg.norm(uhist[-1] - eqn.exact(thist[-1], u_0), numpy.inf)

eqn = linear(numpy.array([[0, 1],
                          [-1, 0]]))
u_0 = numpy.array([.75, 0])

hs = numpy.array([1/2**i for i in range(2, 10)])
pyplot.figure()
method_errors = {}
for name, method in [("Forward Euler", partial(ode_rk_explicit, table=euler)),
                     ("Heun", partial(ode_rk_explicit, table=heun)),
                     ("Bogacki-Shampine", partial(ode_rk_explicit, table=bs3)),
                     ("RK4", partial(ode_rk_explicit, table=rk4)),
                     ("Dormand-Prince", partial(ode_rk_explicit, table=dp45))]:
    errors = []
    for h in hs:
        error = mms(eqn, method, u_0, h=h, T=8)
        errors.append(error)
    method_errors[name] = numpy.asarray(errors)
    pyplot.loglog(hs, errors, ".", label=name)
    
pyplot.loglog(hs, 1.5*hs, label="$\mathcal{O}(h)$")
pyplot.loglog(hs, 2*hs**2, label="$\mathcal{O}(h^2)$")
pyplot.loglog(hs, 0.1*hs**3, label="$\mathcal{O}(h^3)$")
pyplot.loglog(hs, 0.1*hs**4, label="$\mathcal{O}(h^4)$")
pyplot.loglog(hs, 0.002*hs**5, label="$\mathcal{O}(h^5)$")
pyplot.legend()
pyplot.xlabel("h")
pyplot.ylabel("$e_{n,h}$");

### Questions

1. With a timestep of $2^{-7}$, the Dormand-Prince method is already running into machine precision limits. What timestep size would you need for Heun's method to produce the same error?

### Work precision diagrams

This comparison is slightly unfair, since when we are only looking at the error, higher-order methods will clearly perform better for the same timestep. Especially on this oscillatory test problem that has a very smooth solution.

A fairer method of comparison is _work precision_ diagrams. In these, we plot some proxy for the total _cost_ of a method against the obtained error. For explicit methods, the proxy is usually the _number of function evaluations_ (since it avoids arguments about inefficiencies in implementation) though _time to solution_ is also valid.

With this in mind, we can replot our convergence data.

In [ ]:
def work(hs, table, T=15):
    A = table.A
    b = table.b
    stages, _ = A.shape
    if numpy.allclose(A[-1, :], b):
        stages -= 1
    steps = T/hs
    return steps * stages

In [ ]:
cost = partial(work, hs, T=15)
pyplot.figure()
for name, table in [("Forward Euler", euler),
                     ("Heun", heun),
                     ("Bogacki-Shampine", bs3),
                     ("RK4", rk4),
                     ("Dormand-Prince", dp45)]:
    pyplot.loglog(cost(table), method_errors[name], marker="o", label=name)
pyplot.legend()
pyplot.xlabel("Number of function evaluations")
pyplot.ylabel("Error")
pyplot.title("Error vs Cost");

### Questions

1. This looks like the high-order method still always wins. But can you actually increase the step size sufficiently that you can extrapolate to the low-error case?

2. For one of the methods with an embedded method, use the embedded scheme to estimate the error at each step. Compare it to the actual error in the step (computed using the exact solution). Do you see anything interesting?

### Next steps

- Talk about stiff systems: timestep limited by stability of method, rather than desired accuracy.
- Why? Some example chemical reaction problems
- Next: spatial discretisation